In [ ]:
!pip install transformers==4.28.1 torch datasets==2.12.0
!pip install --upgrade huggingface_hub==0.14.1

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 39.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 KB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.9/771.9 KB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 KB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.0/39.0 MB 33.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.7 MB/s eta 0:00:00
     ━━━━━━

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
import torch
import json
import numpy as np
import pandas as pd
from datasets import Dataset, load_metric

In [ ]:
torch.cuda.is_available()

True

In [ ]:
if torch.cuda.is_available():
  print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 4090


In [ ]:
class_list = ["Tanggal Putusan", "Nama Hakim Ketua", "Nama Hakim Anggota", "Nama Panitera",
              "Putusan Hukuman", "Jenis Amar", "Melanggar UU (Pertimbangan Hukum)", "Melanggar UU (Dakwaan)",
              "Jenis Dakwaan", "Tanggal Kejadian", "Nama Saksi", "Tuntutan Hukuman", "Melanggar UU (Tuntutan)",
              "Jenis Perkara", "Nama Terdakwa", "Nama Pengadilan", "Tingkat Kasus", "Nomor Putusan", "Nama Jaksa",
              "Nama Pengacara"]
label_list = ['O']
label_list.extend([_ for __ in [[f'B-{cl}', f'I-{cl}'] for cl in class_list] for _ in __])
print(label_list)

['O', 'B-Tanggal Putusan', 'I-Tanggal Putusan', 'B-Nama Hakim Ketua', 'I-Nama Hakim Ketua', 'B-Nama Hakim Anggota', 'I-Nama Hakim Anggota', 'B-Nama Panitera', 'I-Nama Panitera', 'B-Putusan Hukuman', 'I-Putusan Hukuman', 'B-Jenis Amar', 'I-Jenis Amar', 'B-Melanggar UU (Pertimbangan Hukum)', 'I-Melanggar UU (Pertimbangan Hukum)', 'B-Melanggar UU (Dakwaan)', 'I-Melanggar UU (Dakwaan)', 'B-Jenis Dakwaan', 'I-Jenis Dakwaan', 'B-Tanggal Kejadian', 'I-Tanggal Kejadian', 'B-Nama Saksi', 'I-Nama Saksi', 'B-Tuntutan Hukuman', 'I-Tuntutan Hukuman', 'B-Melanggar UU (Tuntutan)', 'I-Melanggar UU (Tuntutan)', 'B-Jenis Perkara', 'I-Jenis Perkara', 'B-Nama Terdakwa', 'I-Nama Terdakwa', 'B-Nama Pengadilan', 'I-Nama Pengadilan', 'B-Tingkat Kasus', 'I-Tingkat Kasus', 'B-Nomor Putusan', 'I-Nomor Putusan', 'B-Nama Jaksa', 'I-Nama Jaksa', 'B-Nama Pengacara', 'I-Nama Pengacara']


In [ ]:
id2label = {}
label2id = {}
for idx, x in enumerate(label_list):
    label2id[x] = idx
    id2label[idx] = x

In [ ]:
id2label

{0: 'O',
 1: 'B-Tanggal Putusan',
 2: 'I-Tanggal Putusan',
 3: 'B-Nama Hakim Ketua',
 4: 'I-Nama Hakim Ketua',
 5: 'B-Nama Hakim Anggota',
 6: 'I-Nama Hakim Anggota',
 7: 'B-Nama Panitera',
 8: 'I-Nama Panitera',
 9: 'B-Putusan Hukuman',
 10: 'I-Putusan Hukuman',
 11: 'B-Jenis Amar',
 12: 'I-Jenis Amar',
 13: 'B-Melanggar UU (Pertimbangan Hukum)',
 14: 'I-Melanggar UU (Pertimbangan Hukum)',
 15: 'B-Melanggar UU (Dakwaan)',
 16: 'I-Melanggar UU (Dakwaan)',
 17: 'B-Jenis Dakwaan',
 18: 'I-Jenis Dakwaan',
 19: 'B-Tanggal Kejadian',
 20: 'I-Tanggal Kejadian',
 21: 'B-Nama Saksi',
 22: 'I-Nama Saksi',
 23: 'B-Tuntutan Hukuman',
 24: 'I-Tuntutan Hukuman',
 25: 'B-Melanggar UU (Tuntutan)',
 26: 'I-Melanggar UU (Tuntutan)',
 27: 'B-Jenis Perkara',
 28: 'I-Jenis Perkara',
 29: 'B-Nama Terdakwa',
 30: 'I-Nama Terdakwa',
 31: 'B-Nama Pengadilan',
 32: 'I-Nama Pengadilan',
 33: 'B-Tingkat Kasus',
 34: 'I-Tingkat Kasus',
 35: 'B-Nomor Putusan',
 36: 'I-Nomor Putusan',
 37: 'B-Nama Jaksa',
 38: 'I-N

In [ ]:
label2id

{'O': 0,
 'B-Tanggal Putusan': 1,
 'I-Tanggal Putusan': 2,
 'B-Nama Hakim Ketua': 3,
 'I-Nama Hakim Ketua': 4,
 'B-Nama Hakim Anggota': 5,
 'I-Nama Hakim Anggota': 6,
 'B-Nama Panitera': 7,
 'I-Nama Panitera': 8,
 'B-Putusan Hukuman': 9,
 'I-Putusan Hukuman': 10,
 'B-Jenis Amar': 11,
 'I-Jenis Amar': 12,
 'B-Melanggar UU (Pertimbangan Hukum)': 13,
 'I-Melanggar UU (Pertimbangan Hukum)': 14,
 'B-Melanggar UU (Dakwaan)': 15,
 'I-Melanggar UU (Dakwaan)': 16,
 'B-Jenis Dakwaan': 17,
 'I-Jenis Dakwaan': 18,
 'B-Tanggal Kejadian': 19,
 'I-Tanggal Kejadian': 20,
 'B-Nama Saksi': 21,
 'I-Nama Saksi': 22,
 'B-Tuntutan Hukuman': 23,
 'I-Tuntutan Hukuman': 24,
 'B-Melanggar UU (Tuntutan)': 25,
 'I-Melanggar UU (Tuntutan)': 26,
 'B-Jenis Perkara': 27,
 'I-Jenis Perkara': 28,
 'B-Nama Terdakwa': 29,
 'I-Nama Terdakwa': 30,
 'B-Nama Pengadilan': 31,
 'I-Nama Pengadilan': 32,
 'B-Tingkat Kasus': 33,
 'I-Tingkat Kasus': 34,
 'B-Nomor Putusan': 35,
 'I-Nomor Putusan': 36,
 'B-Nama Jaksa': 37,
 'I-Nama 

In [ ]:
def process_input(filename):
  tokens = []
  tags = []
  with open(filename, 'r', encoding="utf8") as f:
    for line in f:
      data = json.loads(line)
      tokens.append(data['text'])
      tags.append(data['text-tags'])

  return pd.DataFrame({'tokens': tokens, "tags": tags})

In [ ]:
dataset_train = process_input("out.train.jsonl")
dataset_test = process_input("out.test.jsonl")
ds_train = Dataset.from_pandas(dataset_train)
ds_test = Dataset.from_pandas(dataset_test)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('indobenchmark/indobert-base-p1')

In [ ]:
# variasi variabel
# stride = 0%, 10%, 25%, 50%
# max_length = 128, 256, 512

def split_tokens(row):
    label_all_data = True
    tokens = tokenizer(
        list(row["tokens"]),
        is_split_into_words=True,
        max_length=128,
        truncation=True,
        return_overflowing_tokens=True,
        return_length=True,
        stride=32,
        padding=True
    )
    token = []
    labels = []

    for i in range(len(tokens.length)):
        m_label = []
        prev_word_idx = None
        token.append(tokens.tokens(i))
        for word_idx in tokens.word_ids(i):
            if word_idx is None:
                m_label.append(-100)
            elif word_idx != prev_word_idx:
                m_label.append(label2id[row["tags"][word_idx]])
            else:
                m_label.append(label2id[row["tags"][word_idx]] if label_all_data else -99)
            prev_word_idx = word_idx
        labels.append(m_label)
    tokens['token'] = token
    tokens['labels'] = labels
    return tokens

In [ ]:
test_tokenized_datasets = ds_test.map(split_tokens)

Map:   0%|          | 0/268 [00:00<?, ? examples/s]

In [ ]:
train_tokenized_datasets = ds_train.map(split_tokens)

Map:   0%|          | 0/695 [00:00<?, ? examples/s]

In [ ]:
train_tokenized_datasets

Dataset({
    features: ['tokens', 'tags', 'input_ids', 'token_type_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping', 'token', 'labels'],
    num_rows: 695
})

In [ ]:
train_tokenized_datasets_flat = train_tokenized_datasets.flatten()

In [ ]:
dropped_col = ['tokens', 'tags', 'length', 'overflow_to_sample_mapping']

In [ ]:
def flatten_pandas_ds(ds):
    cols = list(ds)
    out = {}

    for i in cols:
        d1 = []
        for d1d in ds[i].values.tolist():
            for d2d in d1d.tolist():
                d1.append(d2d)
        out[i] = d1
    return pd.DataFrame(out)

In [ ]:
pd_train_tokenized_datasets = train_tokenized_datasets.to_pandas()
pd_train_tokenized_datasets = pd_train_tokenized_datasets.drop(dropped_col, axis=1)
pd_train_tokenized_datasets_flat = flatten_pandas_ds(pd_train_tokenized_datasets)

In [ ]:
pd_test_tokenized_datasets = test_tokenized_datasets.to_pandas()
pd_test_tokenized_datasets = pd_test_tokenized_datasets.drop(dropped_col, axis=1)
pd_test_tokenized_datasets_flat = flatten_pandas_ds(pd_test_tokenized_datasets)

In [ ]:
train_flat_ds = Dataset.from_pandas(pd_train_tokenized_datasets_flat)
test_flat_ds = Dataset.from_pandas(pd_test_tokenized_datasets_flat)

In [ ]:
!pip install seqeval evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 5.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=a0165e68e158dad86905bd2cfeaa1a313558737e903b46584eb0e5c3ad7e0e98
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


In [ ]:
import evaluate

seqeval = evaluate.load('seqeval')

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer, return_tensors="pt")

In [ ]:
data_collator

DataCollatorForTokenClassification(tokenizer=BertTokenizerFast(name_or_path='indobenchmark/indobert-base-p1', vocab_size=30521, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True), padding=True, max_length=None, pad_to_multiple_of=None, label_pad_token_id=-100, return_tensors='pt')

In [ ]:
# def compute_metrics(p):
#     predictions, labels = p
#     predictions = np.argmax(predictions, axis=2)

#     true_predictions = [
#         [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]
#     true_labels = [
#         [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]

#     results = seqeval.compute(predictions=true_predictions, references=true_labels)
#     return {
#         "precision": results["overall_precision"],
#         "recall": results["overall_recall"],
#         "f1": results["overall_f1"],
#         "accuracy": results["overall_accuracy"],
#     }

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
      if(k not in flattened_results.keys()):
        flattened_results[k+"_f1"]=results[k]["f1"]

    return flattened_results

In [ ]:
model = AutoModelForTokenClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=41, id2label=id2label, label2id=label2id)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
from transformers import EarlyStoppingCallback

In [ ]:
# variasi variabel
# learning rate = 2e-5, 3e-5, 5e-5

args = TrainingArguments(
    output_dir="indobert-base-p1-finetuned-LER-v1",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=16,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end = True,
    metric_for_best_model = 'overall_f1'
)

args.set_optimizer(name="adamw_torch", beta1=0.9)

# args = TrainingArguments(
#     output_dir="training_indobert",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=10,
#     weight_decay=0.01,
#     evaluation_strategy = IntervalStrategy.STEPS,
#     eval_steps = 50,
#     metric_for_best_model = 'f1',
#     load_best_model_at_end=True,
# )

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_flat_ds,
    eval_dataset=test_flat_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,Jenis amar F1,Jenis dakwaan F1,Jenis perkara F1,Melanggar uu (dakwaan) F1,Melanggar uu (pertimbangan hukum) F1,Melanggar uu (tuntutan) F1,Nama hakim anggota F1,Nama hakim ketua F1,Nama jaksa F1,Nama panitera F1,Nama pengacara F1,Nama pengadilan F1,Nama saksi F1,Nama terdakwa F1,Nomor putusan F1,Putusan hukuman F1,Tanggal kejadian F1,Tanggal putusan F1,Tingkat kasus F1,Tuntutan hukuman F1
1,0.022000,0.021020,0.747477,0.731553,0.739429,0.994758,0.731083,0.652233,0.594020,0.465847,0.498096,0.716279,0.849462,0.482599,0.833435,0.839317,0.692308,0.941818,0.778399,0.827056,0.877914,0.498998,0.307407,0.910067,0.917127,0.731656
2,0.013500,0.021522,0.766053,0.772857,0.769440,0.994845,0.874839,0.537994,0.713589,0.489221,0.412567,0.592760,0.849023,0.848161,0.833650,0.863477,0.705202,0.978571,0.778491,0.779357,0.931856,0.609342,0.388670,0.945114,0.937388,0.690476
3,0.011700,0.017783,0.798686,0.785093,0.791831,0.995548,0.881910,0.697740,0.617166,0.577513,0.500385,0.722222,0.868156,0.845109,0.859296,0.858888,0.705521,0.974729,0.810269,0.815608,0.937061,0.468126,0.389610,0.937082,0.961609,0.786813
4,0.009800,0.017310,0.812263,0.798944,0.805549,0.995956,0.871859,0.725784,0.741231,0.642014,0.540581,0.765116,0.863411,0.843792,0.853400,0.848193,0.689855,0.985560,0.820873,0.818946,0.936170,0.538543,0.375000,0.947791,0.959854,0.771310
5,0.008300,0.018085,0.831382,0.807267,0.819147,0.995938,0.876316,0.742120,0.752841,0.632353,0.577498,0.760563,0.888153,0.872943,0.867001,0.873410,0.763975,0.989247,0.826110,0.827533,0.935182,0.589544,0.387426,0.933511,0.970803,0.796178
6,0.007000,0.017951,0.824823,0.810683,0.817692,0.996044,0.917772,0.778860,0.782784,0.653094,0.538192,0.770142,0.892070,0.871728,0.880353,0.891304,0.741935,0.938053,0.831221,0.809195,0.918033,0.638674,0.391913,0.936000,0.950820,0.807818
7,0.006100,0.023405,0.794503,0.824161,0.809061,0.995682,0.916560,0.758916,0.771181,0.598491,0.551499,0.768873,0.893275,0.835556,0.891704,0.871976,0.737179,0.980180,0.793579,0.833078,0.939597,0.621592,0.397898,0.928292,0.968921,0.814332
8,0.005000,0.024918,0.833290,0.796646,0.814556,0.995637,0.900925,0.774963,0.713993,0.612423,0.570992,0.775213,0.879496,0.872340,0.872843,0.887957,0.714286,0.992780,0.829186,0.820233,0.936220,0.565746,0.375092,0.921671,0.961468,0.813596
9,0.003300,0.023350,0.819464,0.811677,0.815552,0.995707,0.913325,0.810440,0.750720,0.642336,0.583159,0.793689,0.883704,0.820755,0.865992,0.860024,0.710280,0.990991,0.820335,0.815729,0.939154,0.657923,0.373938,0.934927,0.963636,0.820399
10,0.002300,0.024431,0.828902,0.826894,0.827897,0.995810,0.934726,0.843407,0.776754,0.641975,0.587842,0.762791,0.898678,0.875083,0.887237,0.880910,0.733728,0.990991,0.838295,0.829755,0.945338,0.650054,0.383442,0.945551,0.963768,0.814655


/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.w

TrainOutput(global_step=117152, training_loss=0.0066172521630323946, metrics={'train_runtime': 7059.5449, 'train_samples_per_second': 132.743, 'train_steps_per_second': 16.595, 'total_flos': 6.123714337998029e+16, 'train_loss': 0.0066172521630323946, 'epoch': 16.0})

## Evaluation

In [ ]:
eval_result = trainer.evaluate()

In [ ]:
!pip install tabulate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


In [ ]:
from tabulate import tabulate

list_result = [['metric', 'score']]
for i in eval_result:
  if i not in ["epoch", 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'eval_loss']:
    list_result.append([i, eval_result[i]])

print(tabulate(list_result))

-----------------------------------------  -------------------
metric                                     score
eval_overall_precision                     0.846731589332315
eval_overall_recall                        0.8262732919254658
eval_overall_f1                            0.8363773537455598
eval_overall_accuracy                      0.9962249814747247
eval_Jenis Amar_f1                         0.93717277486911
eval_Jenis Dakwaan_f1                      0.8421052631578948
eval_Jenis Perkara_f1                      0.7875000000000001
eval_Melanggar UU (Dakwaan)_f1             0.6561461794019933
eval_Melanggar UU (Pertimbangan Hukum)_f1  0.5976207137858642
eval_Melanggar UU (Tuntutan)_f1            0.8076463560334528
eval_Nama Hakim Anggota_f1                 0.9007689491028926
eval_Nama Hakim Ketua_f1                   0.8885959129861569
eval_Nama Jaksa_f1                         0.874310239117106
eval_Nama Panitera_f1                      0.8851674641148326
eval_Nama Pengacara_f1  